In [ ]:
!pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 9.5 MB/s eta 0:00

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
data = load_dataset("FinGPT/fingpt-fiqa_qa", split="train")

README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

(…)-00000-of-00001-ab79bf9300210e98.parquet:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [ ]:
data_df = data.to_pandas()
data_df.head()

,input,output,instruction
0,What is considered a business expense on a bus...,The IRS Guidance pertaining to the subject. I...,"Utilize your financial knowledge, give your an..."
1,Claiming business expenses for a business with...,Yes you can claim your business deductions if ...,Offer your insights or judgment on the input f...
2,Transferring money from One business checking ...,You should have separate files for each of the...,"Based on your financial expertise, provide you..."
3,Having a separate bank account for business/in...,Having a separate checking account for the bus...,Share your insights or perspective on the fina...
4,Having a separate bank account for business/in...,"You don't specify which country you are in, so...",Offer your thoughts or opinion on the input fi...


In [ ]:
data_df["text"] = data_df[["input","output", "instruction"]].apply(lambda x: "###Human:"+x['instruction'] + "\n"+ x["input"] + "\n###Assistant: " +x["output"], axis=1)

In [ ]:
data_df['text'][0]

'###Human:Utilize your financial knowledge, give your answer or opinion to the input question or subject . Answer format is not limited.\nWhat is considered a business expense on a business trip?\n###Assistant: The IRS Guidance pertaining to the subject.  In general the best I can say is your business expense may be deductible.  But it depends on the circumstances and what it is you want to deduct. Travel Taxpayers who travel away from home on business may deduct related   expenses, including the cost of reaching their destination, the cost   of lodging and meals and other ordinary and necessary expenses.   Taxpayers are considered “traveling away from home” if their duties   require them to be away from home substantially longer than an   ordinary day’s work and they need to sleep or rest to meet the demands   of their work. The actual cost of meals and incidental expenses may be   deducted or the taxpayer may use a standard meal allowance and reduced   record keeping requirements. Re

In [ ]:
data = Dataset.from_pandas(data_df)

In [ ]:
data

Dataset({
    features: ['input', 'output', 'instruction', 'text'],
    num_rows: 17110
})

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,   # Use fp16 for computation
    bnb_4bit_use_double_quant=True          # Use double quantization for extra memory savings
)

In [ ]:
model_name = "google/gemma-2-2b"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"  # Automatically maps model layers to available GPUs
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.config.use_cache=False

In [ ]:
model.config.pretraining_tp=1

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=8,                    # Low-rank dimension
    lora_alpha=32,          # Scaling factor for LoRA weights
    target_modules=["q_proj", "v_proj"],  # Target specific modules (adjust based on model architecture)
    lora_dropout=0.1,       # Dropout for LoRA layers
    bias="none",            # No bias updates
    task_type="CAUSAL_LM"   # Task type: causal language modeling
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
           

In [ ]:
training_arguments = TrainingArguments(
        output_dir="gemma-qlora",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        push_to_hub=True,
        report_to="none"
  )

In [ ]:
# Create the SFTTrainer
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=lora_config,
        args=training_arguments,
        tokenizer=tokenizer,
    )

<ipython-input-21-a6f8d4f38e2f>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/17110 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/17110 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/17110 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/17110 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.45 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.79 GiB is free. Process 6299 has 12.95 GiB memory in use. Of the allocated memory 9.86 GiB is allocated by PyTorch, and 2.95 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
! cp -r /content/mistral-finetuned-sentiment-fin /content/drive/MyDrive/

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Abhimanyu9539/mistral-finetuned-sentiment-fin/commit/fe91b74d043a18efa680e58ee5840b30b3a30108', commit_message='End of training', commit_description='', oid='fe91b74d043a18efa680e58ee5840b30b3a30108', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Abhimanyu9539/mistral-finetuned-sentiment-fin', endpoint='https://huggingface.co', repo_type='model', repo_id='Abhimanyu9539/mistral-finetuned-sentiment-fin'), pr_revision=None, pr_num=None)

In [ ]:
!nvidia-smi



Tue Mar 11 09:53:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             30W /   70W |   15010MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/mistral-finetuned-sentiment-fin")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/mistral-finetuned-sentiment-fin",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inputs = tokenizer("""
###Human: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.:
Production levels have been agreed with producers a long time ago , so a fall in consumption will lead to losses .
###Assistant: """, return_tensors="pt").to("cuda")


In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)


###Human: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.: 
Production levels have been agreed with producers a long time ago , so a fall in consumption will lead to losses .
###Assistant: 
negative
0.3079397678375244
